# Prepare Stack NLP from txt to ner app

Neste script iremos transformar txt selecionado num arquivo em formato csv para a marcação de entidades nomeadas.

Usaremos o pacote padrão do python para organizar txt em uma string

Usaremos o pacote NLTK para a detecção de sentenças e para tokenizar

Para o treinamento e uso do Postagger, usaremos o PerceptronTagger do NLTK para treinar o modelo usando o corpus Floresta

In [1]:
import re
import nltk
from nltk import tokenize
from nltk.tag.perceptron import PerceptronTagger
nltk.download('floresta')
from nltk.corpus import floresta

[nltk_data] Downloading package floresta to
[nltk_data]     /home/mardoniofranca/nltk_data...
[nltk_data]   Unzipping corpora/floresta.zip.


Aqui iremos abrir o arquivo txt em uma variavel chamada data


onde tem '../TXT/jardim.txt' você pode mudar o 'jardim' para o nome do txt que você criou no passo 1

In [2]:
with open('../TXT/jardim.txt', 'r') as myfile:
    data = myfile.read()

Ela já está em forma de string, porém cheia de ruído, você pode visualizar ela executando a célula abaixo

In [3]:
data

'Paraibano de nascença, já faz um tem-\npão que compartilho com os conterrâne-\nos da grande nação nordestina a sina da \n“triste partida” ou, em sentido mais po-\nsitivo, a missão de “andar por este país”. \nQuis o desígnio da Providência – os ro-\nmanos apelariam para a senhora Fortu-\nna,  a  deusa  do  destino  –  que,  antes  de \nseguir para o Rio de Janeiro, para ﬁ ns de \nformação proﬁ ssional, eu aportasse por \numa  segunda  vez  em  terras  cearenses \npara uma missão de dois anos. \nLogo  no  início  desta  experiência  mi-\ngratória  –  cheguei  a  Fortaleza  em  feve-\nreiro de 2000, afortunadamente, conheci \numa conterrânea e, através dela, fui aco-\nlhido ﬁ lial e fraternalmente no âmbito de \nsua família. Esta, de raízes paraibanas, \ncuja segunda geração nascera e se criara \nno Rio de Janeiro, experimentou o “êxo-\ndo” para a capital cearense. \nO  seu  nome,  Pilá;  ﬁ lha  segunda  dos \npatriarcas  da  família.  No  aconchego  de \nsua casa, muitas vezes ao redor 

Está cheio de \n e palavras quebradas! iremos corrigir isso usando regex (re) o pacote de expressões regulares do python

lembrando que números de páginas e outros ruídos devem ser removidos manualmente!



primeiro iremos usar a função sub para remover as linhas em branco

In [4]:
data = re.sub('\n\n','\n',data)

depois usaremos a mesma função para juntar as palavras que foram hifenizadas por quebra de texto.

ex:

pe-

dro

dentro da string ela está representada por 'pe-\ndro', se removermos o '-\n' teremos 'pedro' como resultado

In [5]:
data = re.sub('-\n','',data)

finalmente podemos remover as quebras de linhas

In [6]:
data = re.sub('\n','',data)

Pronto! nossa string está limpa! você podem visualizar chamando a variavel data

In [7]:
data

'Paraibano de nascença, já faz um tempão que compartilho com os conterrâneos da grande nação nordestina a sina da “triste partida” ou, em sentido mais positivo, a missão de “andar por este país”. Quis o desígnio da Providência – os romanos apelariam para a senhora Fortuna,  a  deusa  do  destino  –  que,  antes  de seguir para o Rio de Janeiro, para ﬁ ns de formação proﬁ ssional, eu aportasse por uma  segunda  vez  em  terras  cearenses para uma missão de dois anos. Logo  no  início  desta  experiência  migratória  –  cheguei  a  Fortaleza  em  fevereiro de 2000, afortunadamente, conheci uma conterrânea e, através dela, fui acolhido ﬁ lial e fraternalmente no âmbito de sua família. Esta, de raízes paraibanas, cuja segunda geração nascera e se criara no Rio de Janeiro, experimentou o “êxodo” para a capital cearense. O  seu  nome,  Pilá;  ﬁ lha  segunda  dos patriarcas  da  família.  No  aconchego  de sua casa, muitas vezes ao redor da mesa farta,  sempre  regada  com  a  bebida  que ale

## Sentece Detector

Agora iremos separar nossa string em sentenças

a célula abaixo carrega um modelo para o português

In [8]:
sent_tokenizer=nltk.data.load('tokenizers/punkt/portuguese.pickle')

Agora iremos aplicar o modelo a nossa variavel data:

In [9]:
sentences = sent_tokenizer.tokenize(data)

você pode visualizar as sentenças chamando a variavel:

In [10]:
sentences[0:3]

['Paraibano de nascença, já faz um tempão que compartilho com os conterrâneos da grande nação nordestina a sina da “triste partida” ou, em sentido mais positivo, a missão de “andar por este país”.',
 'Quis o desígnio da Providência – os romanos apelariam para a senhora Fortuna,  a  deusa  do  destino  –  que,  antes  de seguir para o Rio de Janeiro, para ﬁ ns de formação proﬁ ssional, eu aportasse por uma  segunda  vez  em  terras  cearenses para uma missão de dois anos.',
 'Logo  no  início  desta  experiência  migratória  –  cheguei  a  Fortaleza  em  fevereiro de 2000, afortunadamente, conheci uma conterrânea e, através dela, fui acolhido ﬁ lial e fraternalmente no âmbito de sua família.']

## Tokenizer

agora iremos separar cada sentença em tokens


para isso criaremos um vetor vazio e salvaremos nele o output da função tokenize.word_tokenize para cada sentença

a função tokenize.word_tokenize recebe um sentença e retorna um vetor de tokens

In [11]:
tokens = []
for sent in sentences:
    tokens.append(tokenize.word_tokenize(sent, language='portuguese'))  

você pode visualizar os tokens chamando a variavel:

In [12]:
tokens[0][0:10]

['Paraibano',
 'de',
 'nascença',
 ',',
 'já',
 'faz',
 'um',
 'tempão',
 'que',
 'compartilho']

## Postag

Uma parte do codigo abaixo foi copiado do site: http://www.nltk.org/howto/portuguese_en.html referente a preparação do corpus floresta. Nele carregamos o corpus, organizamos os tokens e separamos em treino e teste, essa separação é fundamental, pois como vamos treinar nosso próprio modelo de Postag será avaliado os resultados.

In [13]:
def simplify_tag(t):
    if "+" in t:
        return t[t.index("+")+1:]
    else:
        return t


tsents = floresta.tagged_sents()
tsents = [[(w.lower(),simplify_tag(t)) for (w,t) in sent] for sent in tsents if sent]
train = tsents[100:]
test = tsents[:100]

Iremos agora carregar a função que irá fazer o treinamento

Ela é baseada em Perceptrons, um conceito fundamental de inteligência computacional que tenta simular a operação de um neurônio sendo a base das redes neurais modernas.

Referências https://en.wikipedia.org/wiki/Perceptron 

In [14]:
tagger = PerceptronTagger(load=False)

Agora iremos treinar o modelo (...)

In [15]:
tagger.train(train)

Vamos avaliar com a função evaluate. O score varia de 0 a 1, quanto mais proximo de 1 melhor!

In [16]:
tagger.evaluate(test)

0.9520255863539445

Agora iremos usar o nosso modelo para fazer o PoStag em nossas sentenças

iremos salvar em outro vetor chamado sentences_tagged pra não misturar as coisas rs

In [17]:
sentences_tagged = []
for i in tokens:
    sentences_tagged.append(tagger.tag(i))

você pode visualizar o PoStag chamando a variavel:

In [18]:
sentences_tagged[0][0:10]

[('Paraibano', 'n'),
 ('de', 'prp'),
 ('nascença', 'n'),
 (',', ','),
 ('já', 'adv'),
 ('faz', 'v-fin'),
 ('um', 'art'),
 ('tempão', 'n'),
 ('que', 'pron-indp'),
 ('compartilho', 'v-fin')]

## Organizar e Salvar

agora iremos organizar tudo em uma matriz em que a primera coluna é o numero da sentença, a segunda o token, a terceira o PoStag e a quarta a NERtag. Como não taggeamos ainda colocaremos todas igual a 0

In [19]:
matriz = []
for i in range(0,len(sentences_tagged)):
    for j in range(0,len(sentences_tagged[i])):
        matriz.append([i,sentences_tagged[i][j][0],sentences_tagged[i][j][1],0])

você pode visualizar a matriz chamando a variavel:

In [20]:
matriz[0:10]

[[0, 'Paraibano', 'n', 0],
 [0, 'de', 'prp', 0],
 [0, 'nascença', 'n', 0],
 [0, ',', ',', 0],
 [0, 'já', 'adv', 0],
 [0, 'faz', 'v-fin', 0],
 [0, 'um', 'art', 0],
 [0, 'tempão', 'n', 0],
 [0, 'que', 'pron-indp', 0],
 [0, 'compartilho', 'v-fin', 0]]

Agora iremos transformar essa matriz em um dataframe usando o pacote Pandas

dataframe é uma estrutura melhor para visualizar e guardar diferentes tipos de dados

inclusive podemos dizer o que cada coluna é dentro da estrutura

In [21]:
import pandas as pd
df = pd.DataFrame(matriz,columns=['Sentence','Token','PosTag','NERtag'])

você pode visualizar o resultado usando a função head()

In [22]:
df.head()

,Sentence,Token,PosTag,NERtag
0,0,Paraibano,n,0
1,0,de,prp,0
2,0,nascença,n,0
3,0,",",",",0
4,0,já,adv,0


Agora iremos salvar esse dataframe em um csv!

você pode mudar a parte 'jardimNER' para o nome do seu livro

In [23]:
df.to_csv('../NER/jardimNER.csv',index=False)

## Valeu!

### made by Vinicius Sampaio/ Mardônio França from Boitatá